# 라이브러리 및 옵션

In [1]:
import cv2
import numpy as np
import sys
import pandas as pd
import random
import os

In [2]:
print(cv2.__version__)

4.5.3


# 추천할 상품 광고 추출

In [3]:
recommend=pd.read_csv('./recommend.csv')
recommend

,product_id,category_code,brand,price
0,2800103,appliances.kitchen.refrigerators,midea,288.30
1,3200286,appliances.kitchen.meat_grinder,polaris,74.11
2,2800636,appliances.kitchen.refrigerators,indesit,324.08
3,2900634,appliances.kitchen.oven,elenberg,97.79
4,3200013,appliances.kitchen.meat_grinder,panasonic,101.65
5,3200031,appliances.kitchen.meat_grinder,panasonic,76.94
6,6300680,appliances.kitchen.kettle,redmond,33.44
7,3200154,appliances.kitchen.meat_grinder,polaris,64.33
8,6300658,appliances.kitchen.kettle,maxwell,38.35
9,3200280,appliances.kitchen.meat_grinder,moulinex,66.90


In [5]:
random.seed(14)
ad_product_id=str(random.sample(recommend['product_id'].unique().tolist(),1)[0])
ad_product_id

'3200286'

# 영화 + 광고 합성

움직이지 않은 크로마키 + 사진

In [7]:
## 광고 사진 불러오기
path_dir='./ad'
file_list=os.listdir(path_dir)
product_id=ad_product_id+'.jpg'
imgfile=path_dir+'/'+product_id
img=cv2.imread(imgfile,cv2.IMREAD_COLOR)

## 영화 불러오기
cap=cv2.VideoCapture('./video/어바웃타임_크로마키.mp4')
cap2=cv2.VideoCapture('./video/어바웃타임.mp4')
cap_width=int(cap.get(3))
cap_height=int(cap.get(4))

## 저장
fcc=cv2.VideoWriter_fourcc('D','I','V','X')
out=cv2.VideoWriter('./AbouttimeWithAd.avi',fcc,30.0,(cap_width,cap_height))

## 합성 구현
while (cap.isOpened()):
    ret, frame=cap.read()
    if frame is None:
        break
    ret2,frame2=cap2.read()
    
    frame2 = cv2.resize(frame2, (cap_width, cap_height))
    
    ## 마스크 생성
    frame_hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame_threshold=cv2.inRange(frame_hsv,(50,150,0),(80,255,255))
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.erode(frame_threshold,kernel)
    
    ## 크로마키 파악
    contours,_ =cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        point=cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        if len(point)==4:
            pts=point.copy()
            pts=np.reshape(point,(4,-1))

            ## 합성할 이미지 사이즈 조정
            width=max(pts[2][0],pts[3][0])-min(pts[0][0],pts[1][0])
            height=max(pts[1][1],pts[2][1])-min(pts[0][1],pts[3][1])
            if width!=0 and height!=0:

                new=cv2.resize(img, dsize=(width,height), interpolation=cv2.INTER_AREA)

                ## 이미지합성
                origin=frame2[min(pts[0][1],pts[3][1]):max(pts[1][1],pts[2][1]),min(pts[0][0],pts[1][0]):max(pts[2][0],pts[3][0])]
                adver=cv2.addWeighted(origin,0.3,new,0.7,0)
                frame2[min(pts[0][1],pts[3][1]):max(pts[1][1],pts[2][1]),min(pts[0][0],pts[1][0]):max(pts[2][0],pts[3][0])]=adver

    cv2.imshow('frame',frame2)
    out.write(frame2)
    if cv2.waitKey(10)&0xFF==ord('q'):
        break
        
cap.release()
cap2.release()
out.release()
cv2.destroyAllWindows()

# 다른 예시들

움직이는 크로마키 + 사진

In [51]:
## 광고사진 불러오기
imgfile='./images/beer2.jpg'
img=cv2.imread(imgfile,cv2.IMREAD_COLOR)

## 영상 불러오기
cap=cv2.VideoCapture('./video/키싱부스 크로마키 길이편집.mp4')
cap_width=int(cap.get(3))
cap_height=int(cap.get(4))

## 저장
fcc=cv2.VideoWriter_fourcc('D','I','V','X')
out=cv2.VideoWriter('KissingboothWithAd.avi',fcc,30.0,(int(cap_width),int(cap_height)))

## 합성 구현
while (cap.isOpened()):
    ret, frame=cap.read()
    if frame is None:
        break
        
    ## 마스크 생성
    frame_hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame_threshold=cv2.inRange(frame_hsv,(50,150,0),(80,255,255))
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.erode(frame_threshold,kernel)
    
    ## 크로마키 파악
    contours,_ =cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        point=cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        if len(point)==4:
            pts=point.copy()
            pts=np.reshape(point,(4,-1))

            
            ## 합성할 이미지 사이즈 조정
            width=max(pts[2][0],pts[3][0])-min(pts[0][0],pts[1][0])
            height=max(pts[1][1],pts[2][1])-min(pts[0][1],pts[3][1])
            if width!=0 and height!=0:
                new=cv2.resize(img, dsize=(width,height), interpolation=cv2.INTER_AREA)

            ## 이미지합성
            frame[min(pts[0][1],pts[3][1]):max(pts[1][1],pts[2][1]),min(pts[0][0],pts[1][0]):max(pts[2][0],pts[3][0])]=new

    cv2.imshow('frame',frame)
    out.write(frame)
    if cv2.waitKey(10)&0xFF==ord('q'):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

움직이는 크로마키 + 영상

In [43]:
# 녹색 배경 동영상 
mov1 = cv2.VideoCapture('./video/키싱부스 크로마키 길이편집.mp4') 
if not mov1.isOpened(): 
    print('failed!') 
    sys.exit() 

mov_width=int(mov1.get(3))
mov_height=int(mov1.get(4))

# 합성할 광고 동영상 
mov2 = cv2.VideoCapture('./video/iPhone광고.mp4') 
if not mov2.isOpened(): 
    print('failed!') 
    sys.exit() 

mov3= cv2.VideoCapture('./video/iPhone광고.mp4') 
if not mov3.isOpened(): 
    print('failed!') 
    sys.exit() 
    
fps = mov1.get(cv2.CAP_PROP_FPS) 
delay = int(1000 / fps) 
fcc=cv2.VideoWriter_fourcc('D','I','V','X')
out=cv2.VideoWriter('KissingboothWithAdMov.avi',fcc,30.0,(mov_width,mov_height))

while True: 
    ret1, frame1 = mov1.read() 
    if not ret1: 
        break 

    ret2, frame2 = mov2.read() 
    if not ret2: 
        break 
    frame2 = cv2.resize(frame2, (mov_width,mov_height)) # 똑같이 맞춰줌 
    
    ret3, frame3 = mov3.read() 
    if not ret3: 
        break 
        
    ## 마스크와 역마스크 생성
    frame_hsv=cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV)
    frame_threshold=cv2.inRange(frame_hsv,(50,150,0),(80,255,255))
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.erode(frame_threshold,kernel)
    
    ## 크로마키 파악
    contours,_ =cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        point=cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        if len(point)==4:
            pts=point.copy()
            pts=np.reshape(point,(4,-1))

            # 합성할 이미지 사이즈 조정
            width=abs(max(pts[2][0],pts[3][0])-min(pts[0][0],pts[1][0]))
            height=abs(max(pts[1][1],pts[2][1])-min(pts[0][1],pts[3][1]))
            if width!=0 and height!=0:
                new=cv2.resize(frame3, dsize=(width,height), interpolation=cv2.INTER_AREA)

                # 이미지합성
                frame2[min(pts[0][1],pts[3][1]):max(pts[1][1],pts[2][1]),min(pts[0][0],pts[1][0]):max(pts[2][0],pts[3][0])]=new
    
    cv2.copyTo(frame2,mask,frame1)
    cv2.imshow('frame',frame1)
    out.write(frame1)
    if cv2.waitKey(10)&0xFF==ord('q'):
        break
        
mov1.release()
mov2.release()
mov3.release()
out.release()
cv2.destroyAllWindows()

움직이지 않은 크로마키 + 영상

In [28]:
# 녹색 배경 동영상 
mov1 = cv2.VideoCapture('./video/video7.mp4') 
if not mov1.isOpened(): 
    print('failed!') 
    sys.exit() 

mov_width=int(mov1.get(3))
mov_height=int(mov1.get(4))

# 합성할 광고 동영상 
mov2 = cv2.VideoCapture('./video/iPhone광고.mp4') 
if not mov2.isOpened(): 
    print('failed!') 
    sys.exit() 

mov3= cv2.VideoCapture('./video/iPhone광고.mp4') 
if not mov3.isOpened(): 
    print('failed!') 
    sys.exit() 
    
fps = mov1.get(cv2.CAP_PROP_FPS) 
delay = int(1000 / fps) 
fcc=cv2.VideoWriter_fourcc('D','I','V','X')
out=cv2.VideoWriter('NotebookWithAdM.avi',fcc,30.0,(mov_width,mov_height))

while True: 
    ret1, frame1 = mov1.read() 
    if not ret1: 
        break 

    ret2, frame2 = mov2.read() 
    if not ret2: 
        break 
    frame2 = cv2.resize(frame2, (mov_width,mov_height)) # 똑같이 맞춰줌 
    
    ret3, frame3 = mov3.read() 
    if not ret3: 
        break 
        
    ## 마스크와 역마스크 생성
    frame_hsv=cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV)
    frame_threshold=cv2.inRange(frame_hsv,(50,150,0),(80,255,255))
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.erode(frame_threshold,kernel)
    
    ## 크로마키 파악
    contours,_ =cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        point=cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        if len(point)==4:
            pts=point.copy()
            pts=np.reshape(point,(4,-1))

            # 합성할 이미지 사이즈 조정
            width=abs(max(pts[2][0],pts[3][0])-min(pts[0][0],pts[1][0]))
            height=abs(max(pts[1][1],pts[2][1])-min(pts[0][1],pts[3][1]))
            new=cv2.resize(frame3, dsize=(width,height), interpolation=cv2.INTER_AREA)

            # 이미지합성
            frame2[min(pts[0][1],pts[3][1]):max(pts[1][1],pts[2][1]),min(pts[0][0],pts[1][0]):max(pts[2][0],pts[3][0])]=new
    
    cv2.copyTo(frame2,mask,frame1)
    cv2.imshow('frame',frame1)
    out.write(frame1)
    if cv2.waitKey(10)&0xFF==ord('q'):
        break
        
mov1.release()
mov2.release()
mov3.release()
out.release()
cv2.destroyAllWindows()